<a href="https://colab.research.google.com/github/Sidon/pytorch_challenge/blob/master/flower_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
### Install CUDA driver
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [2]:
!pip install -U pillow

Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (5.4.0)


In [0]:
import os
import numpy as np
import json
import torch
from torchvision import datasets as tv_dataset, models
from torchvision import transforms as tv_transforms
from torchvision.utils import make_grid
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys

In [0]:
def mount_gdrive():
  from google.colab import drive
  google_drive_mount = 'gdrive/'
  drive.mount("gdrive/", force_remount=True)
  


In [5]:
colab_kernel = True
if colab_kernel:
  mount_gdrive()


Mounted at gdrive/


In [0]:
!cp 'gdrive/My Drive/colab/pytorch0/torch_helper.py' .
!cp 'gdrive/My Drive/colab/pytorch0/train_model.py' .
!cp 'gdrive/My Drive/colab/pytorch0/util_helper.py' .
!cp 'gdrive/My Drive/colab/pytorch0/cat_to_name.json' .
!cp 'gdrive/My Drive/colab/pytorch0/plt_helper.py' .

In [0]:
# Package imports

if colab_kernel:
  # sys.path.insert(0,'gdrive/My Drive/colab/pytorch0')
  import torch_helper as thelper
  import plt_helper
  import util_helper
  import train_model



In [0]:
!ls

In [0]:
if colab_kernel:
  data_dir = "gdrive/My Drive/colab/flower_data"
  train_dir = data_dir + '/train'
  valid_dir = data_dir + '/valid'

In [0]:
# Creating transforms
normalize = ([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
crop = 224
resize = 256

augment = [tv_transforms.RandomRotation(30), tv_transforms.RandomHorizontalFlip(),
           tv_transforms.CenterCrop(224), tv_transforms.RandomVerticalFlip()]

transforms = {}
transforms['validation'] = thelper.Transforms.validation(resize=resize, crop=crop, normalize=normalize)
transforms['train'] = thelper.Transforms.train(augment=augment, normalize=normalize)

In [0]:
# Creating datasets
dataset = {}
dataset['train'] = thelper.Dataset.dataset(train_dir, transforms['train'])
dataset['validation'] = thelper.Dataset.dataset(valid_dir, transforms['validation'])

In [0]:
# Creating dataloaders
data_loader = {}
data_loader['train'] = thelper.Loaders.loader(dataset['train'])
data_loader['validation'] = thelper.Loaders.loader(dataset['validation'])

In [0]:
# Label mapping
with open('gdrive/My Drive/colab/pytorch0/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
label_map=cat_to_name

Abaixo uma celula para apresentar um batch de imagens mas no colab esta ocorrendo um erro do pill

In [0]:
# # Get a batch of training data
# images, labels = next(iter(data_loader['train']))

# # Get some images from batch
# # images, labels = util_helper.rnd_classes(batch_images, classes_batch_images, label_map, k=4)

# images = images.numpy() # convert images to numpy for display
# labels = labels.numpy()

# # plot the images in the batch, along with the corresponding labels
# fig = plt.figure(figsize=(25, 4))
# for idx in np.arange(20):
#     ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
#     # plt.imshow(np.transpose(images[idx], (1, 2, 0)).astype(np.uint8))
#     plt.imshow(np.transpose(images[idx], (1, 2, 0)))
#     ax.set_title(label_map[str(labels[idx])])


# # # Make a grid from batch
# # grid = make_grid(images)

# # # Plot grid
# # plt_helper.image_show(grid, title = [label[1] for label in labels])

In [18]:
# Create network
out_features = 102
# my_net = thelper.MyNet('resnet152', out_features=out_features)
mount_gdrive()
my_net = thelper.MyNet('vgg16', out_features=out_features)

Mounted at gdrive/


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:33<00:00, 16758625.57it/s]


In [0]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = my_net.create_optmizer()

In [0]:
# Create training object

# model, train_loader, validation_loader, criterion, optimizer, colab_kernel=False, epochs=3,
#              gpu_on=False, model_name = None):


train = train_model.TrainModel(my_net.model, data_loader['train'], data_loader['validation'], criterion, optimizer,   
                               model_name=my_net.trained_model, gpu_on=True )

In [0]:
train.train()

Step:  1 epoch:  0 training_loss:  4.6681437492370605
Step:  2 epoch:  0 training_loss:  9.300402164459229
Step:  3 epoch:  0 training_loss:  13.868295192718506
Step:  4 epoch:  0 training_loss:  18.48255491256714
Step:  5 epoch:  0 training_loss:  23.11095666885376
Step:  6 epoch:  0 training_loss:  27.80910062789917
Step:  7 epoch:  0 training_loss:  32.475882053375244
Step:  8 epoch:  0 training_loss:  37.116429805755615
Step:  9 epoch:  0 training_loss:  41.71677827835083
Step:  10 epoch:  0 training_loss:  46.3550500869751
Step:  11 epoch:  0 training_loss:  50.99484729766846
Step:  12 epoch:  0 training_loss:  55.60764408111572
Step:  13 epoch:  0 training_loss:  60.24363088607788
Step:  14 epoch:  0 training_loss:  64.85625505447388
Step:  15 epoch:  0 training_loss:  69.4524507522583
Step:  16 epoch:  0 training_loss:  74.11461400985718
Step:  17 epoch:  0 training_loss:  78.72568893432617
Step:  18 epoch:  0 training_loss:  83.3608627319336
Step:  19 epoch:  0 training_loss:  